In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import KFold

import random, os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

## Functions

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
    
### SIMPLE BBCE ###

def create_simple_temp_weight(y, hyperparameters, timeSteps=14):
    """Create simple temporal weights for binary cross-entropy based on class imbalance.
    
    Args:
        y (ndarray): Array of binary labels with shape (P, T).
        hyperparameters (dict): Dictionary containing hyperparameters.
        timeSteps (int): Number of time steps (optional, for future use).

    Returns:
        ndarray: Sample weights with the same shape as y.
    """
    
    # Convert the input array to float32
    sample_weights = y.copy().astype(np.float32)
    
    # Count the number of positive and negative samples
    num_positive = np.sum(y == 1)
    num_negative = np.sum(y == 0)
    
    # Calculate the total number of samples
    total_samples = num_positive + num_negative
    
    # Calculate the proportion of each class
    positive_proportion = num_positive / total_samples
    negative_proportion = num_negative / total_samples
    
    # Calculate the weights inversely proportional to the class proportions
    positive_weight = 1.0 / positive_proportion
    negative_weight = 1.0 / negative_proportion
    
    # Assign weights: calculated weight for positive samples and calculated weight for negative samples
    sample_weights[np.where(sample_weights == 1)] = positive_weight
    sample_weights[np.where(sample_weights == 0)] = negative_weight
    
    return sample_weights


def reset_keras(seed=42):
    """Function to ensure that results from Keras models
    are consistent and reproducible across different runs"""
    
    K.clear_session()
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    random.seed(seed)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed)
    # 4. Set `tensorflow` pseudo-random generator at a fixed value
    tf.random.set_seed(seed)


class GRUModel:
    """
    GRUModel class builds and trains a Gated Recurrent Unit (GRU) model
    with specified layers and hyperparameters.
    
    Attributes:
    -----------
    hyperparameters : dict
        A dictionary containing key hyperparameters for model building and training.
        
    Methods:
    --------
    build_model(lr_sch):
        Builds the GRU model with the specified learning rate scheduler.
    train(x_train, y_train, epochs, batch_size, validation_data):
        Trains the built model with the provided training and validation data.
    """
    
    def __init__(self, hyperparameters):
        """
        Initializes the GRUModel with hyperparameters.
        
        Parameters:
        -----------
        hyperparameters : dict
            A dictionary containing key hyperparameters for model building and training.
        """
        self.hyperparameters = hyperparameters
        
    def build_model(self, lr_sch):
        """
        Builds the GRU model with specified learning rate scheduler.
        
        Parameters:
        -----------
        lr_sch : float
            Learning rate for the optimizer during training.
            
        Returns:
        --------
        model : tf.keras.Model
            The compiled GRU model.
        """
        # Define input layer with dynamic shape and masking
        dynamic_input = tf.keras.layers.Input(shape=(self.hyperparameters["timeStep"], self.hyperparameters["layers"][0]))
        masked = tf.keras.layers.Masking(mask_value=self.hyperparameters['maskValue'])(dynamic_input)
        
        # Define GRU layer with specified parameters
        gru_encoder = tf.keras.layers.GRU(
            self.hyperparameters['layers'][1],
            dropout=self.hyperparameters['dropout'],
            return_sequences=False,
            activation='tanh',
            use_bias=True
        )(masked)

        # Define output layer with sigmoid activation function
        output = tf.keras.layers.Dense(1, use_bias=False, activation="sigmoid")(gru_encoder)
        
        # Compile the model with Adam optimizer and custom loss function
        model = tf.keras.Model(dynamic_input, [output])
        my_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_sch)
        model.compile(loss="binary_crossentropy",
                      optimizer=my_optimizer,
                      metrics=['accuracy', 'AUC']
                     )
        
        return model
        
    def train(self, x_train, y_train, x_val, y_val, w1, w2):
        """
        Trains the built model with provided training and validation data.
        
        Parameters:
        -----------
        x_train : numpy array
            Input training data.
        y_train : numpy array
            Target training data.
        epochs : int
            Number of training epochs.
        batch_size : int
            Batch size for training.
        validation_data : tuple
            Tuple containing input and target validation data.
        
        Returns:
        --------
        history : tf.keras.callbacks.History
            A record of training loss values and metrics values at successive epochs.
        model : tf.keras.Model
            The trained GRU model.
        """
        
        model = self.build_model(lr_sch=self.hyperparameters['lr_scheduler'])
        earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      min_delta=self.hyperparameters["mindelta"],
                                                      patience=self.hyperparameters["patience"],
                                                      restore_best_weights=True,
                                                      mode="min")
                    
        history = model.fit(x_train, y_train,
                            validation_data=(x_val, y_val, w2.squeeze()),
                            callbacks=[earlystopping],
                            batch_size=self.hyperparameters['batch_size'], 
                            epochs=self.hyperparameters['epochs'],
                            verbose=0,
                            sample_weight = w1.squeeze())

        return history, model

In [3]:
import numpy as np
from sklearn.model_selection import KFold
from joblib import Parallel, delayed
import multiprocessing

def evaluate_combination(hyperparameters, seed, X_train, y_train, k, l, m, dropout, layers, lr_scheduler):
    hyperparameters_copy = hyperparameters.copy()
    hyperparameters_copy['dropout'] = dropout[k]
    hyperparameters_copy['layers'] = layers[l]
    hyperparameters_copy['lr_scheduler'] = lr_scheduler[m]
    
    v_val_loss = []
    v_hist = []

#     print("\t\tLearning rate:", lr_scheduler[m], ", dropout:", dropout[k], ", layers:", layers[l])
    
    all_patients_train = X_train.shape[0]
    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=seed)
    kf.get_n_splits(all_patients_train)
    for train_index, val_index in kf.split(X_train):
        X_train_cv = X_train[train_index]
        X_val_cv = X_train[val_index]
        y_train_cv = y_train[train_index]
        y_val_cv = y_train[val_index]

        sample_weights_train = create_simple_temp_weight(y_train_cv, hyperparameters_copy, timeSteps=14)
        sample_weights_val = create_simple_temp_weight(y_val_cv, hyperparameters_copy, timeSteps=14)

        reset_keras()
        model = GRUModel(hyperparameters_copy)
        hist, model = model.train(X_train_cv, y_train_cv, X_val_cv, y_val_cv, sample_weights_train, sample_weights_val)

        v_hist.append(hist)
        v_val_loss.append(np.max(hist.history["val_AUC"]))

    metric_dev = np.mean(v_val_loss)
    return (metric_dev, k, l, m, X_train_cv, y_train_cv, X_val_cv, y_val_cv, v_hist)

def myCVGridParallel(hyperparameters, seed, X_train, y_train):
    bestHyperparameters = {'dropout': -1, 'layers': -1, 'lr_scheduler': -1}
    bestMetricDev = -np.inf
    
    lr_scheduler = hyperparameters["lr_scheduler"]
    layers = hyperparameters["layers"]
    dropout = hyperparameters["dropout"]
    
    num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(evaluate_combination)(hyperparameters, seed, X_train, y_train, k, l, m, dropout, layers, lr_scheduler)
        for k in range(len(dropout))
        for l in range(len(layers))
        for m in range(len(lr_scheduler))
    )

    for metric_dev, k, l, m, X_train_cv, y_train_cv, X_val_cv, y_val_cv, v_hist in results:
        if metric_dev > bestMetricDev:
#             print("\t\t\tCambio the best", bestMetricDev, "por metric dev:", metric_dev)
            bestMetricDev = metric_dev
            bestHyperparameters['dropout'] = k
            bestHyperparameters['layers'] = l
            bestHyperparameters['lr_scheduler'] = m
            bestHyperparameters['X_train'] = X_train_cv
            bestHyperparameters['y_train'] = y_train_cv
            bestHyperparameters['X_val'] = X_val_cv
            bestHyperparameters['y_val'] = y_val_cv

    return bestHyperparameters

## Model execution

In [4]:
# Hyperparameters
folders = ["s1"]
seeds = [143, 45, 67]
i = 0
idx_exp = 2
X_train = np.load("../Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/X_train_tensor.npy")

input_shape = X_train.shape[2]
# Select the first 24h - 24 time steps
timeStep = 6
batch_size = 32
epochs = 1000

layer_list = [
    [input_shape, 3, 1],
    [input_shape, 5, 1],
    [input_shape, 8, 1],
    [input_shape, 12, 1],
    [input_shape, 15, 1],
]
dropout = [0, 0.15, 0.3]
lr_scheduler = [1e-1, 1e-2, 1e-3]


hyperparameters = {
    "timeStep": timeStep,
    "maskValue": 666,
    "batch_size": batch_size,
    "epochs": epochs,
    "monitor": "val_loss",
    "mindelta": 0,
    "patience": 30,
    "kfold": 5,
    "dropout": dropout,
    "lr_scheduler": lr_scheduler,
    "layers": layer_list,
    "verbose": 0,
}

In [5]:
metrics_data = []

loss_train = []
loss_dev = []
v_models = []
bestHyperparameters_bySplit = {}
y_pred_by_split = []


for i in range(0,3):

    X_train = np.load("../Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/X_train_tensor.npy")
    y_train = np.load("../Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/y_train_tensor.npy")
    
    X_test = np.load("../Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/X_test_tensor.npy")
    y_test = np.load("../Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/y_test_tensor.npy")

    #GridSearch of hyperparameters and print them   
    bestHyperparameters = myCVGridParallel(hyperparameters, seeds[i], X_train, y_train)
    
    bestHyperparameters_bySplit[str(i)] = bestHyperparameters
    print("\tlr_sch seleccionado:", lr_scheduler[bestHyperparameters["lr_scheduler"]])
    print("\tdropout seleccionado:", dropout[bestHyperparameters["dropout"]])
    print("\tlayers seleccionado:", layer_list[bestHyperparameters["layers"]])
    
    
    besthyperparameters = {
        'timeStep': hyperparameters["timeStep"],
        'maskValue': hyperparameters["maskValue"],
        'batch_size': hyperparameters["batch_size"],
        'epochs': hyperparameters["epochs"],
        'monitor':  hyperparameters["monitor"],
        "mindelta": hyperparameters["mindelta"],
        "patience": hyperparameters["patience"],                    
        "dropout": dropout[bestHyperparameters["dropout"]],
        "layers": layer_list[bestHyperparameters["layers"]],
        "lr_scheduler": lr_scheduler[bestHyperparameters["lr_scheduler"]],                    
        'kfold': hyperparameters["kfold"],
        'verbose': 0
    }
    
    X_train = bestHyperparameters["X_train"]
    y_train = bestHyperparameters["y_train"]
    X_val = bestHyperparameters["X_val"]
    y_val = bestHyperparameters["y_val"]
    
#--- TRY ON TEST -----------------------------------------------------------------------#

    #Reset keras
    reset_keras()
    model = GRUModel(besthyperparameters)
    sample_weights_train = create_simple_temp_weight(y_train, hyperparameters, timeSteps=14)
    sample_weights_val = create_simple_temp_weight(y_val, hyperparameters, timeSteps=14)

    hist, model = model.train(X_train, y_train, X_val, y_val, sample_weights_train, sample_weights_val)

    y_pred = model.predict(x=X_test)
    y_pred = np.reshape(y_pred, (y_pred.size,))
    y_pred_by_split.append(y_pred)
    
    

/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipyke

/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:31: RuntimeWarning: invalid value encountered in scalar divide
/tmp/ipykernel_1244310/555271259.py:32: RuntimeWarning: invalid value encountered in scalar divide


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 6, 15), found shape=(None, 7, 15)

In [ ]:
y_pred_by_split

In [ ]:
for i in range(0,3):
    y_pred_final = y_pred_by_split[i]
    y_test = np.load("../0_Data/splits/App" +str(idx_exp)+ "/"  + folders[i] + "/y_test_tensor.npy")

    #--- METRICS -----------------------------------------------------------------------#     
    accuracy_test = sklearn.metrics.accuracy_score(y_test, np.round(y_pred_final))
    tn, fp, fn, tp = confusion_matrix(y_test, np.round(y_pred_final)).ravel()
    roc = sklearn.metrics.roc_auc_score(y_test, y_pred_final)

    accuracy = accuracy_test
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn) 
    f1score =  (2 * precision * recall) / (precision + recall)

    metrics = {
        "S": i,  
        "TN": tn,
        "TP": tp,
        "FN": fn,
        "FP": fp,
        "ACC": accuracy,
        "SPEC": specificity,
        "PREC": precision,
        "RECALL": recall,
        "F1": f1score,
        "ROC": roc,
    }

In [ ]:
metrics